## Bài toán 1: Đề xuất người dùng với Content-based filtering

### Bước 1: Business Understanding

Vấn đề hiện tại:
Chưa có hệ thống Recommendation System để hỗ trợ người dùng chọn nơi lưu trú phù hợp trên Agoda.

Mục tiêu:
Xây dựng hệ thống đề xuất để hỗ trợ người dùng nhanh chóng chọn được nơi lưu trú phù hợp trên Agoda.
Hệ thống đề xuất sẽ gồm hai mô hình gợi ý chính:
・Content-based filtering: Mô hình này sẽ dựa trên thông tin và đặc điểm của các nơi lưu trú (như dịch vụ, tiện nghi, vị trí, loại phòng) để đề xuất những nơi tương tự mà người dùng có thể quan tâm dựa trên lịch sử tìm kiếm và đánh giá của họ.
・Collaborative filtering: Mô hình này sẽ dựa trên thông tin từ những người dùng khác có hành vi tương tự để đề xuất nơi lưu trú. Nếu nhiều người dùng có cùng sở thích hoặc đánh giá tương tự về các địa điểm, mô hình sẽ sử dụng thông tin này để gợi ý những lựa chọn phù hợp.

### Bước 2: Data Understanding/ Acquire

Đọc và kiểm tra dữ liệu

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from underthesea import word_tokenize, pos_tag, sent_tokenize
import warnings
from gensim import corpora, models, similarities
import re

In [2]:
## Đọc dữ liệu từ các file CSV
hotel_info = pd.read_csv('hotel_info.csv')

In [3]:
hotel_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 740 entries, 0 to 739
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   num                       740 non-null    int64 
 1   Hotel_ID                  740 non-null    object
 2   Hotel_Name                740 non-null    object
 3   Hotel_Rank                740 non-null    object
 4   Hotel_Address             740 non-null    object
 5   Total_Score               740 non-null    object
 6   Location                  413 non-null    object
 7   Cleanliness               412 non-null    object
 8   Service                   373 non-null    object
 9   Facilities                370 non-null    object
 10  Value_for_money           410 non-null    object
 11  Comfort_and_room_quality  51 non-null     object
 12  comments_count            740 non-null    int64 
 13  Hotel_Description         739 non-null    object
dtypes: int64(2), object(12)
me

tổng dữ liệu có 740 dòng;cần hiệu chỉnh đúng kiểu dữ liệu các cột

In [4]:
hotel_info.head()

,num,Hotel_ID,Hotel_Name,Hotel_Rank,Hotel_Address,Total_Score,Location,Cleanliness,Service,Facilities,Value_for_money,Comfort_and_room_quality,comments_count,Hotel_Description
0,1,1_1,Khách sạn Mường Thanh Luxury Nha Trang (Muong ...,5 sao trên 5,"60 Trần Phú, Lộc Thọ, Nha Trang, Việt Nam","8,8","9,4","8,9","8,9","8,7","8,7","8,3",1269,Khách sạn Mường Thanh Luxury Nha Trang - Nơi l...
1,2,1_2,ALPHA BIRD NHA TRANG,4 sao trên 5,"51/19/37 Tue Tinh St, Loc Tho Ward, Nha Trang,...","7,7","7,8","7,6","8,1","7,5","8,1",NaN,337,ALPHA BIRD NHA TRANG - Khách sạn 4.0 sao tại N...
2,3,1_3,Khách sạn Aaron (Aaron Hotel),3.5 sao trên 5,"6Trần Quang Khải, Lộc Thọ, Nha Trang, Việt Nam...","8,5","8,9","8,7","8,8","8,1","8,5",NaN,300,Khách sạn Aaron - Nơi nghỉ dưỡng tuyệt vời tại...
3,4,1_4,Panorama Star Beach Nha Trang,5 sao trên 5,"02 Nguyen Thi Minh Khai, Lộc Thọ, Nha Trang, V...","8,8","9,6","8,9","8,9","8,7","9,0",NaN,814,Panorama Star Beach Nha Trang - Một kỳ nghỉ tu...
4,5,1_5,Khách sạn Balcony Nha Trang (Balcony Nha Trang...,4 sao trên 5,"98B/13 Trần Phú, Lộc Thọ, Nha Trang, Việt Nam","8,4","8,5","8,7","8,5","8,3","8,6","8,7",294,Khách sạn Balcony Nha Trang - Nơi nghỉ dưỡng t...


In [5]:
hotel_info = hotel_info.drop(columns=['num'])

In [6]:
hotel_info.isnull().sum()

Hotel_ID                      0
Hotel_Name                    0
Hotel_Rank                    0
Hotel_Address                 0
Total_Score                   0
Location                    327
Cleanliness                 328
Service                     367
Facilities                  370
Value_for_money             330
Comfort_and_room_quality    689
comments_count                0
Hotel_Description             1
dtype: int64

In [7]:
import pandas as pd
from langdetect import detect

# Function to detect Vietnamese
def is_vietnamese(text):
    try:
        if detect(text) == 'vi':
            return True
        else:
            return False
    except:
        return False

In [8]:
hotel_info_vi= hotel_info[hotel_info['Hotel_Description'].apply(is_vietnamese)]

In [9]:
hotel_info_vi.shape

(586, 13)

In [10]:
hotel_info_nonviet = hotel_info[~hotel_info['Hotel_Description'].apply(is_vietnamese)]

In [11]:
hotel_info_nonviet.shape

(154, 13)

In [12]:
hotel_info_nonviet.head()

,Hotel_ID,Hotel_Name,Hotel_Rank,Hotel_Address,Total_Score,Location,Cleanliness,Service,Facilities,Value_for_money,Comfort_and_room_quality,comments_count,Hotel_Description
56,2_27,"Căn hộ 72 m² 2 phòng ngủ, 2 phòng tắm riêng ở ...",5 sao trên 5,"Vĩnh Phước, Nha Trang, Việt Nam","7,6","8,0","6,0",NaN,NaN,"8,0",NaN,0,"This apartment is Ocean view, 2 bedroom. You c..."
117,5_21,"Chung cư 72 m² 2 phòng ngủ, 2 phòng tắm riêng ...",5 sao trên 5,"Vĩnh Phước, Nha Trang, Việt Nam","8,9","9,2","8,4","9,6","8,0","9,2",NaN,1,• STUNNING VIEWS: Guests will be able to see M...
140,6_22,"Nhà riêng 60 m² 6 phòng ngủ, 1 phòng tắm riêng...",3.5 sao trên 5,"Vĩnh Ngọc, Nha Trang, Việt Nam","2,0","2,0","2,0",NaN,NaN,"2,0",NaN,0,"My's Dung Motel - Apartment, we welcome you to..."
150,11_30,"Căn hộ 42 m² 1 phòng ngủ, 1 phòng tắm riêng ở ...",5 sao trên 5,"Vĩnh Hải, Nha Trang, Việt Nam","8,4","10,0","8,8","7,6","9,2","9,6",NaN,2,My new build apartment located in Nha Trang ci...
156,12_15,"Căn hộ 774 m² 2 phòng ngủ, 2 phòng tắm riêng ở...",5 sao trên 5,"Vĩnh Phước, Nha Trang, Việt Nam",No information,NaN,NaN,NaN,NaN,NaN,NaN,0,"The apartment is spacious and modern, featurin..."


In [13]:
from googletrans import Translator
from langdetect import detect

# Function to translate English text to Vietnamese
def translate_to_vietnamese(text):
    try:
        translator = Translator()
        translation = translator.translate(text, src='en', dest='vi')
        return translation.text
    except Exception as e:
        print(f"Translation error: {e}")
        return text  # Return original text if translation fails

In [14]:
hotel_info_nonviet['Hotel_Description'] = hotel_info_nonviet['Hotel_Description'].apply(translate_to_vietnamese)

Translation error: the JSON object must be str, bytes or bytearray, not NoneType
Translation error: the JSON object must be str, bytes or bytearray, not NoneType


C:\Users\81707\AppData\Local\Temp\ipykernel_17900\1522681454.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_info_nonviet['Hotel_Description'] = hotel_info_nonviet['Hotel_Description'].apply(translate_to_vietnamese)


In [15]:
hotel_info_nonviet.head()

,Hotel_ID,Hotel_Name,Hotel_Rank,Hotel_Address,Total_Score,Location,Cleanliness,Service,Facilities,Value_for_money,Comfort_and_room_quality,comments_count,Hotel_Description
56,2_27,"Căn hộ 72 m² 2 phòng ngủ, 2 phòng tắm riêng ở ...",5 sao trên 5,"Vĩnh Phước, Nha Trang, Việt Nam","7,6","8,0","6,0",NaN,NaN,"8,0",NaN,0,"Căn hộ này là Ocean View, 2 phòng ngủ.Bạn có t..."
117,5_21,"Chung cư 72 m² 2 phòng ngủ, 2 phòng tắm riêng ...",5 sao trên 5,"Vĩnh Phước, Nha Trang, Việt Nam","8,9","9,2","8,4","9,6","8,0","9,2",NaN,1,• Khung cảnh tuyệt đẹp: Khách sẽ có thể nhìn t...
140,6_22,"Nhà riêng 60 m² 6 phòng ngủ, 1 phòng tắm riêng...",3.5 sao trên 5,"Vĩnh Ngọc, Nha Trang, Việt Nam","2,0","2,0","2,0",NaN,NaN,"2,0",NaN,0,"Dung Motel của tôi - Căn hộ, chúng tôi chào đó..."
150,11_30,"Căn hộ 42 m² 1 phòng ngủ, 1 phòng tắm riêng ở ...",5 sao trên 5,"Vĩnh Hải, Nha Trang, Việt Nam","8,4","10,0","8,8","7,6","9,2","9,6",NaN,2,Căn hộ xây dựng mới của tôi nằm ở Trung tâm th...
156,12_15,"Căn hộ 774 m² 2 phòng ngủ, 2 phòng tắm riêng ở...",5 sao trên 5,"Vĩnh Phước, Nha Trang, Việt Nam",No information,NaN,NaN,NaN,NaN,NaN,NaN,0,"Căn hộ rộng rãi và hiện đại, có 2 phòng ngủ th..."


In [16]:
hotel_info_new=pd.concat([hotel_info_vi,hotel_info_nonviet])

In [17]:
hotel_info_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 740 entries, 0 to 709
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Hotel_ID                  740 non-null    object
 1   Hotel_Name                740 non-null    object
 2   Hotel_Rank                740 non-null    object
 3   Hotel_Address             740 non-null    object
 4   Total_Score               740 non-null    object
 5   Location                  413 non-null    object
 6   Cleanliness               412 non-null    object
 7   Service                   373 non-null    object
 8   Facilities                370 non-null    object
 9   Value_for_money           410 non-null    object
 10  Comfort_and_room_quality  51 non-null     object
 11  comments_count            740 non-null    int64 
 12  Hotel_Description         740 non-null    object
dtypes: int64(1), object(12)
memory usage: 80.9+ KB


có 1 dòng là tiếng hàn bị sót->chấp nhận thiếu

dữ liệu null chiếm gần 1/2 tổng dữ liệu,tuy nhiên k ảnh hưởng nên để lại k xóa

In [18]:
# Kiểm tra tỷ lệ phần trăm giá trị thiếu trong mỗi cột
missing_per = hotel_info_new.isnull().sum() / len(hotel_info) * 100
missing_per

Hotel_ID                     0.000000
Hotel_Name                   0.000000
Hotel_Rank                   0.000000
Hotel_Address                0.000000
Total_Score                  0.000000
Location                    44.189189
Cleanliness                 44.324324
Service                     49.594595
Facilities                  50.000000
Value_for_money             44.594595
Comfort_and_room_quality    93.108108
comments_count               0.000000
Hotel_Description            0.000000
dtype: float64

In [19]:
# Kiểm tra giá trị trong cột 'Hotel_Rank'
hotel_info_new['Hotel_Rank'].unique()

array(['5 sao trên 5', '4 sao trên 5', '3.5 sao trên 5', '3 sao trên 5',
       '4.5 sao trên 5', '2.5 sao trên 5', '2 sao trên 5',
       'No information', '1 sao trên 5', '1.5 sao trên 5'], dtype=object)

In [20]:
import re

In [21]:
def extract_number(rank):
    if isinstance(rank, str) and 'sao' in rank and rank.lower() != 'no information':
        match = re.search(r'(\d+(\.\d+)?)\s*sao', rank)
        if match:
            return match.group(1).strip()  # Trả về số trước từ "sao"
    return rank  


In [22]:
hotel_info_new['Hotel_Rank'] = hotel_info_new['Hotel_Rank'].apply(extract_number)

In [23]:
hotel_info_new['Hotel_Rank'].head()

0      5
1      4
2    3.5
3      5
4      4
Name: Hotel_Rank, dtype: object

In [24]:
hotel_info_new['Hotel_Rank'].unique()

array(['5', '4', '3.5', '3', '4.5', '2.5', '2', 'No information', '1',
       '1.5'], dtype=object)

In [25]:
# Đếm số lượng của mỗi giá trị trong cột 'Hotel_Rank_New'
rank_counts = hotel_info_new['Hotel_Rank'].value_counts()
rank_counts

Hotel_Rank
No information    473
5                 123
4                  54
3                  42
3.5                16
4.5                12
2                  12
1.5                 4
2.5                 2
1                   2
Name: count, dtype: int64

rank no information chiếm 1/2

Giá trị No information chiếm số lượng quá nhiều !!!
Kiểm tra dữ liệu thì đa phần là nhà riêng,căn hộ nhỏ
Tuy nhiên vẫn có dữ liệu đánh giá ->chuyển thành Nan

In [26]:
hotel_info_new['Hotel_Rank'] = pd.to_numeric(hotel_info_new['Hotel_Rank'], errors='coerce')

In [27]:
import numpy as np

In [28]:
# Chuyển giá trị 'no information' về NaN
hotel_info_new['Hotel_Rank'] = hotel_info_new['Hotel_Rank'].replace('No information',np.nan)

In [29]:
hotel_info_new['Total_Score'] = hotel_info_new['Total_Score'].replace('No information',np.nan)

In [30]:
hotel_info_new.shape

(740, 13)

In [31]:
# hotel_info = hotel_info.dropna(subset=['Total_Score', 'Location', 'Cleanliness', 'Service', 'Facilities', 'Value_for_money', 'Comfort_and_room_quality'], how='all')
# hotel_info.shape

In [32]:
columns=['Total_Score', 'Location', 'Cleanliness', 'Service', 'Facilities', 'Value_for_money', 'Comfort_and_room_quality']

In [33]:
#Chuyển định đạng cho các cột đánh giá
for column in columns:
    # Thay thế dấu phẩy bằng dấu chấm
    hotel_info_new[column] = hotel_info_new[column].astype(str).str.replace(',', '.')
    # Chuyển đổi cột sang kiểu float
    hotel_info_new[column] = pd.to_numeric(hotel_info_new[column], errors='coerce')

In [34]:
hotel_info_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 740 entries, 0 to 709
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Hotel_ID                  740 non-null    object 
 1   Hotel_Name                740 non-null    object 
 2   Hotel_Rank                267 non-null    float64
 3   Hotel_Address             740 non-null    object 
 4   Total_Score               414 non-null    float64
 5   Location                  413 non-null    float64
 6   Cleanliness               412 non-null    float64
 7   Service                   373 non-null    float64
 8   Facilities                370 non-null    float64
 9   Value_for_money           410 non-null    float64
 10  Comfort_and_room_quality  51 non-null     float64
 11  comments_count            740 non-null    int64  
 12  Hotel_Description         740 non-null    object 
dtypes: float64(8), int64(1), object(4)
memory usage: 80.9+ KB


In [35]:
rank_counts_1 = hotel_info_new['Hotel_Rank'].value_counts()
rank_counts_1

Hotel_Rank
5.0    123
4.0     54
3.0     42
3.5     16
4.5     12
2.0     12
1.5      4
2.5      2
1.0      2
Name: count, dtype: int64

In [36]:
# Kiểm tra số lượng giá trị NaN trong từng cột
nan_counts = hotel_info_new.isna().sum()
nan_counts

Hotel_ID                      0
Hotel_Name                    0
Hotel_Rank                  473
Hotel_Address                 0
Total_Score                 326
Location                    327
Cleanliness                 328
Service                     367
Facilities                  370
Value_for_money             330
Comfort_and_room_quality    689
comments_count                0
Hotel_Description             0
dtype: int64

In [37]:
hotel_info_new.head()

,Hotel_ID,Hotel_Name,Hotel_Rank,Hotel_Address,Total_Score,Location,Cleanliness,Service,Facilities,Value_for_money,Comfort_and_room_quality,comments_count,Hotel_Description
0,1_1,Khách sạn Mường Thanh Luxury Nha Trang (Muong ...,5.0,"60 Trần Phú, Lộc Thọ, Nha Trang, Việt Nam",8.8,9.4,8.9,8.9,8.7,8.7,8.3,1269,Khách sạn Mường Thanh Luxury Nha Trang - Nơi l...
1,1_2,ALPHA BIRD NHA TRANG,4.0,"51/19/37 Tue Tinh St, Loc Tho Ward, Nha Trang,...",7.7,7.8,7.6,8.1,7.5,8.1,NaN,337,ALPHA BIRD NHA TRANG - Khách sạn 4.0 sao tại N...
2,1_3,Khách sạn Aaron (Aaron Hotel),3.5,"6Trần Quang Khải, Lộc Thọ, Nha Trang, Việt Nam...",8.5,8.9,8.7,8.8,8.1,8.5,NaN,300,Khách sạn Aaron - Nơi nghỉ dưỡng tuyệt vời tại...
3,1_4,Panorama Star Beach Nha Trang,5.0,"02 Nguyen Thi Minh Khai, Lộc Thọ, Nha Trang, V...",8.8,9.6,8.9,8.9,8.7,9.0,NaN,814,Panorama Star Beach Nha Trang - Một kỳ nghỉ tu...
4,1_5,Khách sạn Balcony Nha Trang (Balcony Nha Trang...,4.0,"98B/13 Trần Phú, Lộc Thọ, Nha Trang, Việt Nam",8.4,8.5,8.7,8.5,8.3,8.6,8.7,294,Khách sạn Balcony Nha Trang - Nơi nghỉ dưỡng t...


In [38]:
# Tạo một hàm để lấy tên phường
def extract_ward(address):
    # Tách địa chỉ thành danh sách các phần
    parts = address.split(', ')
    # Tìm vị trí của 'Nha Trang' từ dưới lên
    try:
        idx = parts[::-1].index('Nha Trang')
        # Tính toán chỉ số của tên phường
        ward_name = parts[-(idx + 2)].strip()  # Lấy phần trước 'Nha Trang'
        return ward_name
    except ValueError:
        return None

In [39]:
hotel_info_new['Ward_Name'] = hotel_info_new['Hotel_Address'].apply(extract_ward)

In [40]:
hotel_info_new['Ward_Name'].head()

0    Lộc Thọ
1    Lộc Thọ
2    Lộc Thọ
3    Lộc Thọ
4    Lộc Thọ
Name: Ward_Name, dtype: object

In [41]:
# Lấy danh sách các giá trị duy nhất từ cột 'Ward Name'
unique_wards = hotel_info_new['Ward_Name'].unique()
unique_wards

array(['Lộc Thọ', 'Vĩnh Hải', 'Vĩnh Phước', 'Cam Hải Đông', 'Tân Lập',
       'Vĩnh Hòa', 'Vĩnh Nguyên', 'Xương Huân', 'Vĩnh Trường',
       'Vạn Thắng', 'Dốc Lết', 'Phước Hòa', 'Hòn Tre', 'Cam Ranh',
       'Vịnh Ninh Vân', 'Ngọc Hiệp', 'Phước Long', 'Cam Đức',
       'Vĩnh Thạnh', 'Vĩnh Hiệp', 'Biển Bãi Dài', 'Vịnh Vân Phong',
       'Vĩnh Ngọc', 'Phước Hạ', 'Vĩnh Thái', 'Vạn Ninh',
       'Huyện Diên Khánh', 'Vĩnh Phương', 'Phước Hải', 'Phước Tiến'],
      dtype=object)

Gensim và cosine_similarity là hai khái niệm khác nhau trong xử lý ngôn ngữ tự nhiên (NLP), mặc dù cả hai đều có thể liên quan đến việc xử lý và so sánh văn bản.

Gensim:là một thư viện mã nguồn mở cho Python, được sử dụng để xử lý ngôn ngữ tự nhiên, biểu diễn văn bản bằng vector (word embeddings), và tìm kiếm tương đồng văn bản.
Chức năng: Gensim hỗ trợ nhiều mô hình học máy như Word2Vec, Doc2Vec, và các kỹ thuật giảm chiều dữ liệu như Latent Semantic Indexing (LSI) và Latent Dirichlet Allocation (LDA). Nó thường được sử dụng để biến đổi văn bản thô thành các vector số, mà sau đó có thể được sử dụng cho các tác vụ phân tích và mô hình hóa khác nhau.

In [42]:
hotel_info_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 740 entries, 0 to 709
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Hotel_ID                  740 non-null    object 
 1   Hotel_Name                740 non-null    object 
 2   Hotel_Rank                267 non-null    float64
 3   Hotel_Address             740 non-null    object 
 4   Total_Score               414 non-null    float64
 5   Location                  413 non-null    float64
 6   Cleanliness               412 non-null    float64
 7   Service                   373 non-null    float64
 8   Facilities                370 non-null    float64
 9   Value_for_money           410 non-null    float64
 10  Comfort_and_room_quality  51 non-null     float64
 11  comments_count            740 non-null    int64  
 12  Hotel_Description         740 non-null    object 
 13  Ward_Name                 740 non-null    object 
dtypes: float64(8), 

In [43]:
# Chuyển tất cả các giá trị trong các cột thành chuỗi và sau đó thực hiện phép nối
hotel_info_new['Content'] = hotel_info_new[[ 'Hotel_Name', 'Hotel_Description','Ward_Name']].fillna('').astype(str).agg(' '.join, axis=1)


In [44]:
hotel_info_new['Content'].head()

0    Khách sạn Mường Thanh Luxury Nha Trang (Muong ...
1    ALPHA BIRD NHA TRANG ALPHA BIRD NHA TRANG - Kh...
2    Khách sạn Aaron (Aaron Hotel) Khách sạn Aaron ...
3    Panorama Star Beach Nha Trang Panorama Star Be...
4    Khách sạn Balcony Nha Trang (Balcony Nha Trang...
Name: Content, dtype: object

In [45]:
# word_tokenize
hotel_info_new["Content_wt"]=hotel_info_new["Content"].apply(lambda x: word_tokenize(x, format="text"))

In [46]:
hotel_info_new[["Content", "Content_wt"]].head()

,Content,Content_wt
0,Khách sạn Mường Thanh Luxury Nha Trang (Muong ...,Khách_sạn Mường_Thanh_Luxury Nha_Trang ( Muong...
1,ALPHA BIRD NHA TRANG ALPHA BIRD NHA TRANG - Kh...,ALPHA BIRD NHA_TRANG ALPHA_BIRD NHA_TRANG - Kh...
2,Khách sạn Aaron (Aaron Hotel) Khách sạn Aaron ...,Khách_sạn Aaron ( Aaron_Hotel ) Khách_sạn Aaro...
3,Panorama Star Beach Nha Trang Panorama Star Be...,Panorama Star_Beach Nha_Trang Panorama Star_Be...
4,Khách sạn Balcony Nha Trang (Balcony Nha Trang...,Khách_sạn Balcony Nha_Trang ( Balcony Nha_Tran...


In [47]:
# Tokenize(split) the sentences into words
content_gem = [[text for text in x.split()] for x in hotel_info_new.Content_wt]

In [48]:
# # Danh sách các cụm từ cần giữ lại
# special_tokens = ['5km', '30phut']

# # Hàm tách từ với các cụm từ đặc biệt
# def tokenize_with_special(text):
#     # Thay thế các cụm từ đặc biệt bằng placeholder
#     for token in special_tokens:
#         text = text.replace(token, f'__{token}__')
#     # Tách từ và khôi phục các cụm từ đặc biệt
#     tokens = re.findall(r'\b\w+\b', text)
#     # Khôi phục các cụm từ đặc biệt
#     tokens = [token if token not in ['__' + st + '__' for st in special_tokens] else st for st in special_tokens]
#     return tokens

In [49]:
# content_ = [tokenize_with_special(x) if isinstance(x, str) else [] for x in hotel_info['Content_wt']]

In [50]:
# def tokenize_text(text):
#      if isinstance(text, str):
#             return text.split()
#      return [text]

In [51]:
# # Áp dụng hàm tokenize_text cho cột Content_wt
# hotel_info['content_gem_new'] = hotel_info['Content_wt'].apply(tokenize_text)

In [52]:
# hotel_info['content_gem_new'].head()

In [53]:
content_gem[:1]

[['Khách_sạn',
  'Mường_Thanh_Luxury',
  'Nha_Trang',
  '(',
  'Muong_Thanh_Luxury',
  'Nha_Trang_Hotel',
  ')',
  'Khách_sạn',
  'Mường_Thanh_Luxury',
  'Nha_Trang',
  '-',
  'Nơi',
  'lưu_trú',
  'tuyệt_vời',
  'tại',
  'Nha_Trang',
  'Khách_sạn',
  'Mường_Thanh_Luxury',
  'Nha_Trang',
  'nằm',
  'ở',
  'trung_tâm',
  'thành_phố',
  'Nha_Trang',
  ',',
  'Việt_Nam',
  ',',
  'là',
  'một',
  'khách_sạn',
  '5',
  'sao',
  'đẳng_cấp',
  '.',
  'Với',
  'vị_trí',
  'thuận_lợi',
  ',',
  'khách_sạn',
  'này',
  'chỉ',
  'cách',
  'trung_tâm',
  'thành_phố',
  '0.2',
  'km',
  'và',
  'cách',
  'sân_bay',
  'khoảng',
  '39',
  'phút',
  'đi',
  'xe',
  '.',
  'Khách_sạn',
  'Mường_Thanh_Luxury',
  'Nha_Trang',
  'được',
  'xây_dựng',
  'vào',
  'năm',
  '2014',
  'và',
  'hiện',
  'có',
  'tổng_cộng',
  '458',
  'phòng',
  '.',
  'Với',
  'số_lượng',
  'phòng',
  'lớn',
  'như_vậy',
  ',',
  'khách_sạn',
  'này',
  'đảm_bảo',
  'sẽ',
  'đáp_ứng',
  'được',
  'nhu_cầu',
  'lưu_trú',
  'củ

In [54]:
len(content_gem)

740

Kqua tách đc 740 từ và cụm từ

In [55]:
# Danh sách các ký tự đặc biệt cần loại bỏ
special_characters = ['', ' ', ',', '.', '...', '-', ':', ';', '?', '%', '(', ')', '+', '/', "'", '&', '!']

# Xử lý ký tự đặc biệt
content_gem_re = [[t.lower() for t in text if t not in special_characters]for text in content_gem]

In [56]:
content_gem_re[:1]

[['khách_sạn',
  'mường_thanh_luxury',
  'nha_trang',
  'muong_thanh_luxury',
  'nha_trang_hotel',
  'khách_sạn',
  'mường_thanh_luxury',
  'nha_trang',
  'nơi',
  'lưu_trú',
  'tuyệt_vời',
  'tại',
  'nha_trang',
  'khách_sạn',
  'mường_thanh_luxury',
  'nha_trang',
  'nằm',
  'ở',
  'trung_tâm',
  'thành_phố',
  'nha_trang',
  'việt_nam',
  'là',
  'một',
  'khách_sạn',
  '5',
  'sao',
  'đẳng_cấp',
  'với',
  'vị_trí',
  'thuận_lợi',
  'khách_sạn',
  'này',
  'chỉ',
  'cách',
  'trung_tâm',
  'thành_phố',
  '0.2',
  'km',
  'và',
  'cách',
  'sân_bay',
  'khoảng',
  '39',
  'phút',
  'đi',
  'xe',
  'khách_sạn',
  'mường_thanh_luxury',
  'nha_trang',
  'được',
  'xây_dựng',
  'vào',
  'năm',
  '2014',
  'và',
  'hiện',
  'có',
  'tổng_cộng',
  '458',
  'phòng',
  'với',
  'số_lượng',
  'phòng',
  'lớn',
  'như_vậy',
  'khách_sạn',
  'này',
  'đảm_bảo',
  'sẽ',
  'đáp_ứng',
  'được',
  'nhu_cầu',
  'lưu_trú',
  'của',
  'du_khách',
  'một_cách',
  'thoải_mái',
  'và',
  'tiện_nghi',


In [57]:
STOP_WORD_FILE = 'files/vietnamese-stopwords.txt'

In [58]:
with open(STOP_WORD_FILE, 'r', encoding='utf-8') as file:
    stop_words = file.read()

stop_words = stop_words.split('\n')

In [59]:
content_gem_re = [[t for t in text if not t in stop_words] for text in content_gem_re] # stopword

In [60]:
content_gem_re[:1]

[['mường_thanh_luxury',
  'muong_thanh_luxury',
  'mường_thanh_luxury',
  'tuyệt_vời',
  'mường_thanh_luxury',
  'trung_tâm',
  'thành_phố',
  '5',
  'đẳng_cấp',
  'thuận_lợi',
  'trung_tâm',
  'thành_phố',
  '0.2',
  'sân_bay',
  '39',
  'xe',
  'mường_thanh_luxury',
  '2014',
  '458',
  'phòng',
  'số_lượng',
  'phòng',
  'đáp_ứng',
  'nhu_cầu',
  'du_khách',
  'thoải_mái',
  'tiện_nghi',
  'phòng',
  '02',
  '00',
  'phòng',
  '12',
  '00',
  'pm._điều',
  'cho_phép',
  'du_khách',
  'thư_giãn',
  'tận_hưởng',
  'tiện_ích',
  'dịch_vụ',
  'tuyệt_vời',
  'cho_phép',
  'trẻ_em',
  '6',
  '11',
  'miễn_phí',
  'thuận_lợi',
  'gia_đình',
  'trẻ',
  'tọa_lạc',
  'du_khách',
  'tiện_nghi',
  'giải_trí',
  'tuyệt_vời',
  'cửa_hàng',
  'đa_dạng',
  'đáp_ứng',
  'nhu_cầu',
  'mua_sắm',
  'trải_nghiệm',
  'trò_chơi',
  'độc_đáo',
  'thân_thiện',
  'thưởng_thức',
  'đồ',
  'thơm',
  'ngon',
  'thư_giãn',
  'dạo',
  'làm_đẹp',
  'chăm_sóc',
  'sức_khỏe',
  'salon',
  'spa',
  'tận_hưởng',
  'li

In [61]:
# Nén tất cả các từ từ các danh sách thành một danh sách duy nhất
uni_list = [word for sublist in content_gem_re for word in sublist]

In [62]:
# Tạo DataFrame với mỗi từ trên một dòng
df_uni = pd.DataFrame({'word': uni_list})

In [63]:
# Xuất DataFrame ra file CSV với mỗi từ trên một dòng
df_uni.to_csv('content_gem_re_flat.csv', index=False, header=False)

xuất ra csv lọc trùng,kiểm tra bổ sung stop words
dữ liệu không có ý nghĩa nhiều nên quyết định xóa

In [64]:
content_gem_re = [[re.sub('[0-9]+','', e) for e in text] for text in content_gem_re]

In [65]:
content_gem_re[:1]

[['mường_thanh_luxury',
  'muong_thanh_luxury',
  'mường_thanh_luxury',
  'tuyệt_vời',
  'mường_thanh_luxury',
  'trung_tâm',
  'thành_phố',
  '',
  'đẳng_cấp',
  'thuận_lợi',
  'trung_tâm',
  'thành_phố',
  '.',
  'sân_bay',
  '',
  'xe',
  'mường_thanh_luxury',
  '',
  '',
  'phòng',
  'số_lượng',
  'phòng',
  'đáp_ứng',
  'nhu_cầu',
  'du_khách',
  'thoải_mái',
  'tiện_nghi',
  'phòng',
  '',
  '',
  'phòng',
  '',
  '',
  'pm._điều',
  'cho_phép',
  'du_khách',
  'thư_giãn',
  'tận_hưởng',
  'tiện_ích',
  'dịch_vụ',
  'tuyệt_vời',
  'cho_phép',
  'trẻ_em',
  '',
  '',
  'miễn_phí',
  'thuận_lợi',
  'gia_đình',
  'trẻ',
  'tọa_lạc',
  'du_khách',
  'tiện_nghi',
  'giải_trí',
  'tuyệt_vời',
  'cửa_hàng',
  'đa_dạng',
  'đáp_ứng',
  'nhu_cầu',
  'mua_sắm',
  'trải_nghiệm',
  'trò_chơi',
  'độc_đáo',
  'thân_thiện',
  'thưởng_thức',
  'đồ',
  'thơm',
  'ngon',
  'thư_giãn',
  'dạo',
  'làm_đẹp',
  'chăm_sóc',
  'sức_khỏe',
  'salon',
  'spa',
  'tận_hưởng',
  'liệu_pháp',
  'massage',


In [66]:
len(content_gem_re)

740

số phần tử trong list bằng số dòng của hotel_infor_new,thêm vào dataframe

In [67]:
content_gem_re_str = [' '.join(text) for text in content_gem_re]

In [68]:
hotel_info_new['Content_gem_re'] = content_gem_re_str

In [69]:
hotel_info_new.head()

,Hotel_ID,Hotel_Name,Hotel_Rank,Hotel_Address,Total_Score,Location,Cleanliness,Service,Facilities,Value_for_money,Comfort_and_room_quality,comments_count,Hotel_Description,Ward_Name,Content,Content_wt,Content_gem_re
0,1_1,Khách sạn Mường Thanh Luxury Nha Trang (Muong ...,5.0,"60 Trần Phú, Lộc Thọ, Nha Trang, Việt Nam",8.8,9.4,8.9,8.9,8.7,8.7,8.3,1269,Khách sạn Mường Thanh Luxury Nha Trang - Nơi l...,Lộc Thọ,Khách sạn Mường Thanh Luxury Nha Trang (Muong ...,Khách_sạn Mường_Thanh_Luxury Nha_Trang ( Muong...,mường_thanh_luxury muong_thanh_luxury mường_th...
1,1_2,ALPHA BIRD NHA TRANG,4.0,"51/19/37 Tue Tinh St, Loc Tho Ward, Nha Trang,...",7.7,7.8,7.6,8.1,7.5,8.1,NaN,337,ALPHA BIRD NHA TRANG - Khách sạn 4.0 sao tại N...,Lộc Thọ,ALPHA BIRD NHA TRANG ALPHA BIRD NHA TRANG - Kh...,ALPHA BIRD NHA_TRANG ALPHA_BIRD NHA_TRANG - Kh...,alpha bird alpha_bird ._sao alpha_bird ._sao t...
2,1_3,Khách sạn Aaron (Aaron Hotel),3.5,"6Trần Quang Khải, Lộc Thọ, Nha Trang, Việt Nam...",8.5,8.9,8.7,8.8,8.1,8.5,NaN,300,Khách sạn Aaron - Nơi nghỉ dưỡng tuyệt vời tại...,Lộc Thọ,Khách sạn Aaron (Aaron Hotel) Khách sạn Aaron ...,Khách_sạn Aaron ( Aaron_Hotel ) Khách_sạn Aaro...,aaron aaron_hotel aaron nghỉ_dưỡng tuyệt_vời a...
3,1_4,Panorama Star Beach Nha Trang,5.0,"02 Nguyen Thi Minh Khai, Lộc Thọ, Nha Trang, V...",8.8,9.6,8.9,8.9,8.7,9.0,NaN,814,Panorama Star Beach Nha Trang - Một kỳ nghỉ tu...,Lộc Thọ,Panorama Star Beach Nha Trang Panorama Star Be...,Panorama Star_Beach Nha_Trang Panorama Star_Be...,panorama star_beach panorama star_beach kỳ_ngh...
4,1_5,Khách sạn Balcony Nha Trang (Balcony Nha Trang...,4.0,"98B/13 Trần Phú, Lộc Thọ, Nha Trang, Việt Nam",8.4,8.5,8.7,8.5,8.3,8.6,8.7,294,Khách sạn Balcony Nha Trang - Nơi nghỉ dưỡng t...,Lộc Thọ,Khách sạn Balcony Nha Trang (Balcony Nha Trang...,Khách_sạn Balcony Nha_Trang ( Balcony Nha_Tran...,balcony balcony balcony nghỉ_dưỡng tuyệt_vời b...


In [70]:
content_gem_re = hotel_info_new['Content_gem_re'].apply(lambda x: x.split())

In [71]:
# Obtain the number of features based on dictionary: Use corpora.Dictionary
dictionary = corpora.Dictionary(content_gem_re)

In [72]:
# List of features in dictionary
dictionary.token2id# ánh xạ từ đến chỉ số trong từ điển.(Danh sách các tính năng (từ) trong từ điển)

{'!_lộc_thọ': 0,
 '.': 1,
 '/': 2,
 'agoda': 3,
 'amart': 4,
 'an_toàn': 5,
 'anh_thư': 6,
 'bao_gồm': 7,
 'bay': 8,
 'bia_tiệp': 9,
 'biển': 10,
 'buffet': 11,
 'bánh': 12,
 'bãi': 13,
 'bãi_biển': 14,
 'bóng_chuyền': 15,
 'bơi': 16,
 'bơi_lội': 17,
 'bắc_thành': 18,
 'bể': 19,
 'bồn': 20,
 'bữa': 21,
 'cam_kết': 22,
 'cam_ranh': 23,
 'cao': 24,
 'central_park': 25,
 'cho_phép': 26,
 'chuon_chuon': 27,
 'chuyên_nghiệp': 28,
 'chuyến': 29,
 'chân': 30,
 'chùa': 31,
 'chú_ý': 32,
 'chăm_sóc': 33,
 'chất_lượng': 34,
 'chỗ': 35,
 'chợ': 36,
 'cuộc_sống': 37,
 'cà_phê': 38,
 'cá_sấu': 39,
 'cách_tiện': 40,
 'công_cộng': 41,
 'công_sức': 42,
 'công_viên': 43,
 'công_viên_nước': 44,
 'căng_thẳng': 45,
 'cơ_sở': 46,
 'cảnh_quan': 47,
 'cửa_hàng': 48,
 'da': 49,
 'deluxe': 50,
 'deluxe_family': 51,
 'di_chuyển': 52,
 'diện_tích': 53,
 'double_bed': 54,
 'dr._tooth': 55,
 'du_khách': 56,
 'duy_trì': 57,
 'dạo': 58,
 'dễ_dàng': 59,
 'dịch_vụ': 60,
 'dịp': 61,
 'dừng': 62,
 'executive_suite': 63,

In [73]:
# Numbers of features (word) in dictionary
feature_cnt = len(dictionary.token2id)

In [74]:
feature_cnt#Số lượng từ khác nhau trong từ điển

7212

In [75]:
# Obtain corpus based on dictionary (dense matrix)
corpus = [dictionary.doc2bow(text) for text in content_gem_re]

In [76]:
corpus[0] # id, so lan xuat hien cua token trong van ban/ san pham((id từ, số lần xuất hiện).)

[(0, 1),
 (1, 4),
 (2, 1),
 (3, 3),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 2),
 (8, 1),
 (9, 1),
 (10, 7),
 (11, 1),
 (12, 1),
 (13, 5),
 (14, 1),
 (15, 1),
 (16, 2),
 (17, 1),
 (18, 1),
 (19, 2),
 (20, 1),
 (21, 2),
 (22, 1),
 (23, 2),
 (24, 2),
 (25, 1),
 (26, 2),
 (27, 1),
 (28, 4),
 (29, 2),
 (30, 2),
 (31, 1),
 (32, 1),
 (33, 4),
 (34, 1),
 (35, 1),
 (36, 1),
 (37, 1),
 (38, 4),
 (39, 1),
 (40, 1),
 (41, 3),
 (42, 1),
 (43, 1),
 (44, 1),
 (45, 1),
 (46, 1),
 (47, 1),
 (48, 5),
 (49, 1),
 (50, 4),
 (51, 1),
 (52, 3),
 (53, 6),
 (54, 2),
 (55, 1),
 (56, 34),
 (57, 1),
 (58, 2),
 (59, 8),
 (60, 26),
 (61, 1),
 (62, 1),
 (63, 1),
 (64, 1),
 (65, 1),
 (66, 3),
 (67, 1),
 (68, 1),
 (69, 10),
 (70, 7),
 (71, 1),
 (72, 1),
 (73, 1),
 (74, 1),
 (75, 1),
 (76, 1),
 (77, 4),
 (78, 1),
 (79, 1),
 (80, 1),
 (81, 1),
 (82, 1),
 (83, 1),
 (84, 1),
 (85, 1),
 (86, 1),
 (87, 1),
 (88, 1),
 (89, 1),
 (90, 1),
 (91, 2),
 (92, 2),
 (93, 1),
 (94, 10),
 (95, 9),
 (96, 10),
 (97, 1),
 (98, 1),
 (99, 1),
 (100

Mô Hình TF-IDF đo lường mức độ tương đồng

In [77]:
# Use TF-IDF Model to process corpus, obtaining index
tfidf = models.TfidfModel(corpus)
# tính toán sự tương tự trong ma trận thưa thớt
index = similarities.SparseMatrixSimilarity(tfidf[corpus],num_features = feature_cnt)
# ma tran: n x n
#mỗi hàng của ma trận tương ứng với một văn bản trong corpus và mỗi cột tương ứng với sự tương tự với các văn bản khác.

In [78]:
# Chuyển ma trận tương tự thành DataFrame
df_hotel_info = pd.DataFrame(index)
df_hotel_info

,0,1,2,3,4,5,6,7,8,9,...,730,731,732,733,734,735,736,737,738,739
0,1.000000,0.599165,0.559707,0.474832,0.547772,0.556914,0.469327,0.424140,0.447638,0.109056,...,0.052252,0.024286,0.040318,0.053407,0.010121,0.022669,0.027628,0.031372,0.057771,0.041096
1,0.599165,1.000000,0.624457,0.531522,0.595558,0.583082,0.513470,0.458374,0.492242,0.134715,...,0.052226,0.021849,0.053685,0.053380,0.026231,0.023630,0.021026,0.032701,0.060738,0.055422
2,0.559707,0.624457,1.000000,0.522605,0.594138,0.587573,0.488415,0.459167,0.499738,0.132026,...,0.040446,0.028319,0.054712,0.041340,0.011024,0.017416,0.039236,0.024102,0.044063,0.039391
3,0.474832,0.531522,0.522605,1.000000,0.524201,0.515319,0.390324,0.368640,0.442788,0.096993,...,0.045903,0.027691,0.054184,0.051882,0.013086,0.023951,0.020301,0.033145,0.056871,0.038519
4,0.547772,0.595558,0.594138,0.524201,1.000000,0.542605,0.468775,0.433235,0.475121,0.133756,...,0.047034,0.025972,0.059361,0.048073,0.015575,0.016396,0.021649,0.022691,0.050864,0.036008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
735,0.022669,0.023630,0.017416,0.023951,0.016396,0.028577,0.013675,0.025057,0.055210,0.012374,...,0.039629,0.037936,0.032801,0.040505,0.022834,1.000000,0.012043,0.709168,0.118646,0.001269
736,0.027628,0.021026,0.039236,0.020301,0.021649,0.019247,0.018588,0.016041,0.015830,0.025050,...,0.034398,0.068006,0.030062,0.035158,0.018775,0.012043,1.000000,0.017988,0.025886,0.013530
737,0.031372,0.032701,0.024102,0.033145,0.022691,0.043472,0.018925,0.035049,0.077220,0.017127,...,0.056176,0.061843,0.045980,0.057417,0.032727,0.709168,0.017988,1.000000,0.175062,0.002229
738,0.057771,0.060738,0.044063,0.056871,0.050864,0.037159,0.034912,0.049254,0.076400,0.022592,...,0.051662,0.065571,0.049531,0.052804,0.030275,0.118646,0.025886,0.175062,1.000000,0.007314


In [79]:
hotel_info_new['Content_gem_re']

0      mường_thanh_luxury muong_thanh_luxury mường_th...
1      alpha bird alpha_bird ._sao alpha_bird ._sao t...
2      aaron aaron_hotel aaron nghỉ_dưỡng tuyệt_vời a...
3      panorama star_beach panorama star_beach kỳ_ngh...
4      balcony balcony balcony nghỉ_dưỡng tuyệt_vời b...
                             ...                        
703    nhà_riêng  m²  phòng ngủ  phòng vĩnh_phước ans...
705    nhà_riêng  m²  phòng ngủ  phòng phước_tiến joi...
707    nhà_riêng  m²  phòng ngủ  phòng vĩnh_phước căn...
708    nhà_riêng  m²  phòng ngủ  phòng vĩnh_phước sea...
709     m²  phòng ngủ  phòng phước_hạ z_camp chào_mừn...
Name: Content_gem_re, Length: 740, dtype: object

In [80]:
# Tạo từ điển
dictionary = corpora.Dictionary(hotel_info_new['Content_gem_re'].apply(lambda x: x.split()))

In [81]:
# Tạo corpus từ danh sách token
corpus = [dictionary.doc2bow(text.split()) for text in hotel_info_new['Content_gem_re']]

In [82]:
# Tạo mô hình TF-IDF
tfidf = models.TfidfModel(corpus)

In [83]:
# Tạo ma trận tương tự
index = similarities.SparseMatrixSimilarity(tfidf[corpus], num_features=len(dictionary.token2id))

In [84]:
# # Hàm gợi ý khách sạn tương tự
# def recommend_similar_hotels(hotel_id, top_n=5):
#     # Lấy nội dung của khách sạn đã chọn và chuyển thành danh sách các token
#     view_content = hotel_info_new.loc[hotel_info_new['Hotel_ID'] == hotel_id, 'Content_gem_re'].values
#     if len(view_content) == 0:
#         return "Khách sạn không tồn tại."

#     # Chia nhỏ chuỗi thành danh sách các token
#     view_tokens = view_content[0].split()  
#     view_bow = dictionary.doc2bow(view_tokens)
#     view_tfidf = tfidf[view_bow]
#     sims = index[view_tfidf]

  
#     sim_df = pd.DataFrame({
#         'Hotel_ID': hotel_info_new['Hotel_ID'],
#         'Hotel_Name': hotel_info_new['Hotel_Name'],  
#         'Similarity': sims
#     })

#     # Bỏ qua khách sạn đã chọn (similarity = 1)
#     sim_df = sim_df[sim_df['Hotel_ID'] != hotel_id]

#     # Sắp xếp và lấy top N khách sạn tương tự
#     sim_df = sim_df.sort_values(by='Similarity', ascending=False)
#     return sim_df.head(top_n)


In [95]:
import time

In [96]:
# Hàm gợi ý khách sạn tương tự
def recommend_similar_hotels(hotel_id, top_n=5):
    # Bắt đầu đo thời gian
    start_time = time.time()
    
    # Lấy nội dung của khách sạn đã chọn và chuyển thành danh sách các token
    view_content = hotel_info_new.loc[hotel_info_new['Hotel_ID'] == hotel_id, 'Content_gem_re'].values
    if len(view_content) == 0:
        return "Khách sạn không tồn tại."

    # Chia nhỏ chuỗi thành danh sách các token
    view_tokens = view_content[0].split()  
    view_bow = dictionary.doc2bow(view_tokens)
    view_tfidf = tfidf[view_bow]
    sims = index[view_tfidf]

    # Tạo DataFrame với thông tin khách sạn và độ tương tự
    sim_df = pd.DataFrame({
        'Hotel_ID': hotel_info_new['Hotel_ID'],
        'Hotel_Name': hotel_info_new['Hotel_Name'],  
        'Similarity': sims
    })

    # Bỏ qua khách sạn đã chọn (similarity = 1)
    sim_df = sim_df[sim_df['Hotel_ID'] != hotel_id]

    # Sắp xếp và lấy top N khách sạn tương tự
    sim_df = sim_df.sort_values(by='Similarity', ascending=False)
    
    # Kết thúc đo thời gian
    end_time = time.time()
    
    # Tính toán thời gian đã trôi qua
    elapsed_time = end_time - start_time
    print(f"Time taken to generate recommendations: {elapsed_time} seconds")
    
    return sim_df.head(top_n)

In [99]:
# Ví dụ sử dụng hàm để gợi ý khách sạn tương tự cho Hotel_ID '1_1'
recommended_hotels = recommend_similar_hotels('1_1', top_n=5)
recommended_hotels 

Time taken to generate recommendations: 0.008515596389770508 seconds


,Hotel_ID,Hotel_Name,Similarity
49,2_20,Khách sạn V Nha Trang (V Hotel Nha Trang),0.613084
626,8_15,Khách sạn Nhi Phi (Nhi Phi Hotel Nha Trang),0.602465
11,1_12,Khách sạn AZURA (Azura Hotel),0.599192
1,1_2,ALPHA BIRD NHA TRANG,0.599165
37,2_8,Dubai Nha Trang Hotel,0.593799


Cosine Similarity:là một biện pháp đo lường độ tương đồng giữa hai vector trong không gian vector. Nó đặc biệt hữu ích trong việc so sánh các tài liệu văn bản khi chúng được biểu diễn dưới dạng vector.
 Chức năng: Cosine similarity đo góc giữa hai vector và trả về giá trị từ -1 đến 1. Giá trị 1 biểu thị hai vector hoàn toàn tương tự (cùng hướng), 0 biểu thị chúng không liên quan (vuông góc), và -1 biểu thị chúng hoàn toàn trái ngược. Trong ngữ cảnh văn bản, các vector có thể là biểu diễn TF-IDF, word embeddings, hoặc bất kỳ vector nào khác được tạo ra từ văn bản.

In [87]:
from numpy import dot
from numpy.linalg import norm
X = [1,2]
Y = [2,2]
cos_sim = dot(X,Y) / (norm(X)*norm(Y))
print(cos_sim)

0.9486832980505138


In [88]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Vector hóa nội dung
vectorizer = TfidfVectorizer(analyzer='word', stop_words=stop_words)
tfidf_matrix = vectorizer.fit_transform(hotel_info_new['Content_wt'])

# Tính toán độ tương đồng
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

c:\Users\81707\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['tháng_8'] not in stop_words.
  warnings.warn(


In [89]:
df_show = pd.DataFrame(cosine_sim)
df_show

,0,1,2,3,4,5,6,7,8,9,...,730,731,732,733,734,735,736,737,738,739
0,1.000000,0.769290,0.744842,0.683524,0.733216,0.715847,0.699491,0.651399,0.661054,0.262651,...,0.131276,0.071487,0.093409,0.117481,0.041471,0.086769,0.078232,0.123099,0.127747,0.090236
1,0.769290,1.000000,0.782350,0.717280,0.757689,0.741583,0.725940,0.658779,0.690864,0.280141,...,0.129040,0.067721,0.117403,0.129297,0.057470,0.092007,0.067761,0.129202,0.136989,0.107316
2,0.744842,0.782350,1.000000,0.710001,0.763440,0.745391,0.699089,0.681268,0.706135,0.282296,...,0.106593,0.072300,0.101668,0.105332,0.044607,0.087542,0.078012,0.114751,0.117079,0.084002
3,0.683524,0.717280,0.710001,1.000000,0.709446,0.711604,0.620130,0.595303,0.655445,0.245911,...,0.130511,0.082882,0.115549,0.137817,0.063282,0.102982,0.070875,0.142549,0.163586,0.088578
4,0.733216,0.757689,0.763440,0.709446,1.000000,0.708438,0.685681,0.640900,0.673388,0.272509,...,0.119588,0.076173,0.105001,0.118333,0.050278,0.077446,0.063515,0.109086,0.125598,0.082721
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
735,0.086769,0.092007,0.087542,0.102982,0.077446,0.102232,0.068803,0.084011,0.131356,0.077014,...,0.098251,0.112239,0.061822,0.098446,0.057138,1.000000,0.047547,0.722883,0.232110,0.011756
736,0.078232,0.067761,0.078012,0.070875,0.063515,0.064113,0.057947,0.056224,0.060042,0.073420,...,0.072753,0.108105,0.059012,0.072898,0.041034,0.047547,1.000000,0.072627,0.074933,0.027247
737,0.123099,0.129202,0.114751,0.142549,0.109086,0.147088,0.098323,0.115966,0.172760,0.114704,...,0.138150,0.177840,0.087345,0.138425,0.081195,0.722883,0.072627,1.000000,0.337769,0.018774
738,0.127747,0.136989,0.117079,0.163586,0.125598,0.126515,0.099082,0.128336,0.155135,0.106549,...,0.129198,0.172916,0.099763,0.129455,0.097121,0.232110,0.074933,0.337769,1.000000,0.024342


In [90]:
# # Hàm đề xuất khách sạn
# # # với mỗi sản phẩm, lấy nums sản phẩm tương quan nhất
# def get_recommendations(hotel_id, cosine_sim=cosine_sim, nums=5):
#     idx = hotel_info_new.index[hotel_info_new['Hotel_ID'] == hotel_id][0]
#     sim_scores = list(enumerate(cosine_sim[idx]))
#     sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
#     sim_scores = sim_scores[1:nums+1]  # Lấy 5 khách sạn tương tự nhất
#     hotel_indices = [i[0] for i in sim_scores]
#     return hotel_info_new['Hotel_Name'].iloc[hotel_indices]

In [91]:
import time

In [92]:
# def get_recommendations(hotel_id, cosine_sim, nums=5):

#     # Bắt đầu đo thời gian
#     start_time = time.time()

#     # Lấy chỉ số của khách sạn đã chọn
#     idx = hotel_info_new.index[hotel_info_new['Hotel_ID'] == hotel_id][0]
    
#     # Tính toán độ tương tự giữa khách sạn đã chọn và tất cả các khách sạn khác
#     sim_scores = list(enumerate(cosine_sim[idx]))
    
#     # Sắp xếp độ tương tự giảm dần
#     sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
#     # Lấy các khách sạn tương tự nhất (bỏ qua khách sạn đã chọn)
#     sim_scores = sim_scores[1:nums+1]
    
#     # Lấy chỉ số của các khách sạn tương tự
#     hotel_indices = [i[0] for i in sim_scores]
    
#     # Tạo DataFrame với các thông tin cần thiết
#     recommendations = pd.DataFrame({
#         'Hotel_ID': hotel_info_new['Hotel_ID'].iloc[hotel_indices].values,
#         'Hotel_Name': hotel_info_new['Hotel_Name'].iloc[hotel_indices].values,
#         'Similarity': [i[1] for i in sim_scores]
#     })
    
#     return recommendations

In [93]:
import pandas as pd
import time

def get_recommendations(hotel_id, cosine_sim, nums=5):
    # Bắt đầu đo thời gian
    start_time = time.time()
    
    # Lấy chỉ số của khách sạn đã chọn
    idx = hotel_info_new.index[hotel_info_new['Hotel_ID'] == hotel_id][0]
    
    # Tính toán độ tương tự giữa khách sạn đã chọn và tất cả các khách sạn khác
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Sắp xếp độ tương tự giảm dần
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Lấy các khách sạn tương tự nhất (bỏ qua khách sạn đã chọn)
    sim_scores = sim_scores[1:nums+1]
    
    # Lấy chỉ số của các khách sạn tương tự
    hotel_indices = [i[0] for i in sim_scores]
    
    # Tạo DataFrame với các thông tin cần thiết
    recommendations = pd.DataFrame({
        'Hotel_ID': hotel_info_new['Hotel_ID'].iloc[hotel_indices].values,
        'Hotel_Name': hotel_info_new['Hotel_Name'].iloc[hotel_indices].values,
        'Similarity': [i[1] for i in sim_scores]
    })
    
    # Kết thúc đo thời gian
    end_time = time.time()
    
    # Tính toán thời gian đã trôi qua
    elapsed_time = end_time - start_time
    print(f"Time taken to generate recommendations: {elapsed_time} seconds")
    
    return recommendations


In [94]:
recommended_hotels = get_recommendations('1_1', cosine_sim, nums=5)
print(recommended_hotels)

Time taken to generate recommendations: 0.004230499267578125 seconds
  Hotel_ID                                 Hotel_Name  Similarity
0     2_20  Khách sạn V Nha Trang (V Hotel Nha Trang)    0.791281
1      2_8                      Dubai Nha Trang Hotel    0.775656
2      6_9                    Angella Hotel Nha Trang    0.772511
3     1_12              Khách sạn AZURA (Azura Hotel)    0.769504
4      1_2                       ALPHA BIRD NHA TRANG    0.769290


Kết Quả Tương Tự:
Khách sạn ALPHA BIRD NHA TRANG (1_2) và Khách sạn AZURA (1_12) xuất hiện trong cả hai bảng kết quả
Gensim có thể đưa ra kết quả thấp hơn do sự khác biệt trong cách tính toán và tiền xử lý dữ liệu.
Khách sạn Angella Hotel Nha Trang (6_9) không xuất hiện trong kết quả từ Gensim nhưng có mặt trong kết quả từ Cosine Similarity. Điều này cho thấy rằng các phương pháp có thể khám phá các mối tương quan khác nhau.

Độ Chính Xác:
Gensim: Dựa vào mô hình dựa trên tf-idf và từ điển, Gensim có thể tính toán độ tương tự dựa trên các từ khóa trong nội dung. Điểm số thấp hơn có thể do cách mô hình học và biểu diễn dữ liệu không hoàn toàn khớp với cách tính toán khác.
Cosine Similarity: Đây là phương pháp dựa trên sự tương đồng về góc giữa các vector, thường cho kết quả chính xác hơn trong nhiều trường hợp, nhất là khi dữ liệu được chuẩn hóa tốt.

Độ Phù Hợp:
Gensim: Có thể cho ra các kết quả có phần khác biệt hơn trong việc đề xuất khách sạn tương tự, vì nó phụ thuộc vào cách mà mô hình học từ dữ liệu và từ điển.
Cosine Similarity: Thường đánh giá sự tương đồng dựa trên các đặc trưng toàn diện hơn và có thể phản ánh đúng hơn mức độ tương tự giữa các đối tượng.

Tính Đồng Nhất:
Gensim: Nếu dữ liệu không được chuẩn bị tốt hoặc các từ khóa không đại diện chính xác cho nội dung, kết quả có thể không nhất quán.
Cosine Similarity: Cung cấp kết quả đồng nhất hơn khi dữ liệu được chuẩn bị đúng cách và các vector tương tự được chuẩn hóa.
Gensim: Kết quả có thể cho thấy các khách sạn tương tự nhưng có thể không bao quát tất cả các khách sạn phù hợp.
Cosine Similarity: Có thể đưa ra các đề xuất chính xác hơn và bao quát hơn.

Run time

gensim:0.0085 (s)

cosine:0.0042 (s)

tốc độ xử lý của cosine nhanh hơn

->chọn cosine để áp dụng